# Model Denemeleri

### Önce HuggingFace'ten, hali hazırda tamamen türkçe veriler üzerinde eğitilmiş BerTurk modeli denenecek

In [15]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForPreTraining.from_pretrained("dbmdz/bert-base-turkish-cased")

In [16]:
sd_hf = model.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [17]:
torch.equal(sd_hf["bert.embeddings.word_embeddings.weight"], sd_hf["cls.predictions.decoder.weight"]) # tying kontrolü

True

In [18]:
sd_hf["bert.embeddings.word_embeddings.weight"].data_ptr() == sd_hf["cls.predictions.decoder.weight"].data_ptr()  # tying kontrolü

True

In [19]:
torch.equal(sd_hf["cls.predictions.decoder.bias"], sd_hf["cls.predictions.bias"]) # tying kontrolü

True

In [20]:
sd_hf["cls.predictions.decoder.bias"].data_ptr() == sd_hf["cls.predictions.bias"].data_ptr()  # tying kontrolü

True

In [21]:
inputs = tokenizer("Selamlar efendim", return_tensors="pt")
outputs = model(**inputs)

In [22]:
outputs.prediction_logits.shape, outputs.seq_relationship_logits.shape

(torch.Size([1, 4, 32000]), torch.Size([1, 2]))

In [23]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [24]:
from model import BertForPreTraining, BertConfig

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'BertForPreTraining' from 'model' (unknown location)

In [ ]:
default_config = BertConfig()
custom_model = BertForPreTraining(default_config)

In [ ]:
custom_sd = custom_model.state_dict()

assert len(custom_sd) == len(sd_hf)

for custom, hf in zip(custom_sd.items(), sd_hf.items()):
    k, v = custom
    k_hf, v_hf = hf
    assert k == k_hf, f"{k} : {k_hf}"
    assert v.shape == v_hf.shape, f"{k} : {v.shape} != {v_hf.shape}"
    print(k, v.shape, "--------", k_hf, v_hf.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768]) -------- bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768]) -------- bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768]) -------- bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768]) -------- bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768]) -------- bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768]) -------- bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768]) -------- bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768]) -

In [ ]:
model_from_hf = BertForPreTraining.from_pretrained()
mm = model_from_hf.state_dict()

loading weights from pretrained bert: dbmdz/bert-base-turkish-cased


In [ ]:
from transformers import pipeline

torch.random.manual_seed(42)
fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?") 

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [ ]:
fill_masker("ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı") 

[{'score': 0.9745508432388306,
  'token': 22777,
  'token_str': 'Muhammet',
  'sequence': 'ışınlanma teknolojisini Muhammet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.016110757365822792,
  'token': 7397,
  'token_str': 'Muhammed',
  'sequence': 'ışınlanma teknolojisini Muhammed Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0018351072212681174,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'ışınlanma teknolojisini Mehmet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.00045096001122146845,
  'token': 6001,
  'token_str': 'Hasan',
  'sequence': 'ışınlanma teknolojisini Hasan Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0004167473816778511,
  'token': 6391,
  'token_str': 'Cem',
  'sequence': 'ışınlanma teknolojisini Cem Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'}]

In [ ]:
# auto-detect device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"[INFO] using device: {device}")

[INFO] using device: cuda


In [ ]:
# WITHOUT PAD TOKEN
from model import FillMaskPipeline, IsNextPipeline
torch.random.manual_seed(42)
model_from_hf.to(device)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "[MASK] yaptın?"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: [MASK] yaptın? ----> Top 5 Predictions: {'token_str': ['ne', 'neler', 'nasıl', 'neden', 'Ne'], 'score': ['0.5882', '0.0835', '0.0642', '0.0283', '0.0223']}


In [ ]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Hasan', 'Cem'], 'score': ['0.9561', '0.0241', '0.0035', '0.0011', '0.0007']}


In [ ]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Hasan', 'Cem'], 'score': ['0.9561', '0.0241', '0.0035', '0.0011', '0.0007']}


In [ ]:
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}


In [ ]:
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="multinomial")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'Mustafa', 'Bey', 'Ali', 'osman'], 'score': ['0.0415', '0.0253', '0.0073', '0.0242', '0.0032']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['yavaş', 'hoşuma', 'ileri', 'yanlış', 'hoşuna'], 'score': ['0.0113', '0.0057', '0.0342', '0.0133', '0.0210']}


In [ ]:
# random initialized, so garbage output expected
fill_mask_custom = FillMaskPipeline(model=BertForPreTraining(BertConfig()), tokenizer=tokenizer, strategy="greedy")    
fill_mask_custom(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."]) 

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['istiyordum', 'mesafesinde', '##D', 'Kit', '##ırsın'], 'score': ['0.0002', '0.0002', '0.0003', '0.0003', '0.0002']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['Ster', 'Hon', '##idge', 'mesafesinde', 'Kan'], 'score': ['0.0002', '0.0003', '0.0002', '0.0003', '0.0003']}


In [ ]:
nsp_pipeline = IsNextPipeline(model=model_from_hf, tokenizer=tokenizer)

In [ ]:
nsp_pipeline([["Çocuk sahibi çift sayısında inanılmaz bir artış var.", "Uzaya ilk Bekiroğ Reis çıkmıştır"],
              ["Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.", "Bir şekilde kabloları ayarlamak gerekli"],
              ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", "Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!"] ])

Text: ['Çocuk sahibi çift sayısında inanılmaz bir artış var.', 'Uzaya ilk Bekiroğ Reis çıkmıştır'] Predictions ----> isNext: 0.072, notNext: 0.928
Text: ['Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.', 'Bir şekilde kabloları ayarlamak gerekli'] Predictions ----> isNext: 1.000, notNext: 0.000
Text: ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", 'Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!'] Predictions ----> isNext: 0.056, notNext: 0.944


In [ ]:
from bert_implementation_tr.data.data_aux import load_xy_shard, visualize_sample, ModelInput, VisualizeModelInput, get_tokenizer
from model import *
import torch

# auto-detect device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"[INFO] using device: {device}")

%load_ext autoreload
%autoreload 2

[INFO] using device: cuda


In [ ]:
# device = "cpu" # let's override it right now

In [ ]:
random_model = BertForPreTraining(BertConfig())
random_model = random_model.to(device)

In [ ]:
# Toplam parametre sayısını hesapla
total_params = sum(p.numel() for p in random_model.parameters())
print("Toplam Parametre Sayısı:", total_params)

Toplam Parametre Sayısı: 111243010


In [ ]:
# Parametre başına byte cinsinden boyut (örneğin, float32 için 4 byte)
param_size = 4

# Toplam boyutu hesapla (byte cinsinden)
total_memory_bytes = total_params * param_size

# GB cinsine çevir
total_memory_gb = total_memory_bytes / (1024 ** 3)

print("Model Bellek Kullanımı (GB):", total_memory_gb)

Model Bellek Kullanımı (GB): 0.4144125059247017


In [ ]:
#one_sample_model_input = ModelInput(
#                             input_ids=model_inputs_shard_0.input_ids[0],
#                             labels=model_inputs_shard_0.labels[0],
#                             attention_mask=model_inputs_shard_0.attention_mask[0],
#                             token_type_ids=model_inputs_shard_0.token_type_ids[0],
#                             next_sentence_label=model_inputs_shard_0.next_sentence_label[0]
#                         )
#visualize_sample(VisualizeModelInput(one_sample_model_input, show_ids=True, show_attention_and_segment=True))

In [ ]:
temp_dict = ModelInput.from_numpy_to_tensors_dict(load_xy_shard(0, block_size=256)[0:1], block_size=256)

In [ ]:
temp_dict = { k: v.to(device) for k, v in temp_dict.items()}

In [ ]:
out = random_model(**temp_dict)

In [ ]:
print("total loss: ", out.loss.item()) 
print("seq_relationship_logits: ", out.seq_relationship_logits) 
print("prediction_logits: ", out.prediction_logits)

total loss:  11.218903541564941
seq_relationship_logits:  tensor([[0.0442, 0.1998]], device='cuda:0', grad_fn=<AddmmBackward0>)
prediction_logits:  tensor([[[ 1.2935, -0.4987,  0.3163,  ..., -0.2023, -0.0688, -0.9404],
         [ 0.8961, -0.6119, -0.9890,  ...,  0.4870, -0.4408, -0.4997],
         [ 0.7256, -0.3088,  0.2867,  ...,  0.5798, -0.7951, -0.3199],
         ...,
         [ 0.1811, -0.0490, -1.1911,  ...,  1.0088,  0.1699, -0.0295],
         [ 0.8136,  0.0698, -0.1525,  ...,  0.9332,  0.6340, -0.6499],
         [ 0.2951, -0.3336, -0.3732,  ...,  0.3542,  0.2385, -0.3002]]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [ ]:
# nextseqlabel en sağa
# labels'da clas tokeni pad token olacak (ya da buradaki tüm pad tokenlar -100 olacak: tek bir ce yeter o zaman)
# benim kendi vocab'ım 32001 tane aslında (sonradan "..." eklediğim için) buna bi çözüm
# şimdilik yukarıdaki problemleri geçici çözeceğim

# a.convert_tokens_to_ids("..."), a.convert_tokens_to_ids(".")

In [ ]:
#import torch
#
## Cihazı belirle
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
## GPU'da bfloat16 desteği var mı?
#if device.type == "cuda":
#    bfloat16_supported = torch.cuda.get_device_capability(device) >= (8, 0)  # A100 (ampere) ve sonrası bfloat16 destekler
#    print("bfloat16 desteği:", bfloat16_supported)
#else:
#    print("GPU mevcut değil.")

bfloat16 desteği: True


In [ ]:
# Test for dataloader
from bert_implementation_tr.data.data_aux import DataLoaderCustom

%load_ext autoreload
%autoreload 2

#train_loader = DataLoaderCustom(batch_size=16, block_size=256, split="train", device="cuda", verbose=False)
val_loader = DataLoaderCustom(batch_size=32, block_size=512, split="train", device="cuda")
acc = 0
for batch in range(1000):
    return_a = val_loader.next_batch()
    if return_a is not None:
        acc += return_a["input_ids"].shape[0]
        print(acc)
    if return_a is None:
        print("val finished, no update anymore")
        
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

split: train

data shards directory: bert_implementation_tr/data/xy_shards_256
block size: 256
batch size: 32
total number of shards: 35
total number of tokens: 350999552
total number of samples: 1371092
-------------------------------------------------------------------------
1 batch: 8192 tokens
1 epoch: 42846 batches
1 epoch: 350999552 tokens
1 shard: ~10028558 tokens
1 shard: ~1224 batches
-------------------------------------------------------------------------

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
304

In [ ]:
val_loader.current_position_in_shard, val_loader.current_shard_id

(3200, 35)

In [ ]:
val_loader.reset()

In [ ]:
(step % pretrain_config["val_check_interval"] == 0) or last_step

In [ ]:
40 == 0 or False

False

In [ ]:
85 % 55 == 0 or False

False

In [ ]:
4 % 55 == 0 or False

False

In [ ]:
False or False

False

In [ ]:
# WITHOUT PAD TOKEN
import torch
from model import FillMaskPipeline, BertForPreTraining
from transformers import AutoTokenizer, AutoModel
from bert_implementation_tr.data.data_aux import DataLoaderCustom


%load_ext autoreload
%autoreload 2

device = "cuda"
torch.random.manual_seed(42)


# model_hf = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")
# model_hf.to(device)
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model_from_hf = BertForPreTraining.from_pretrained()
model_from_hf.to(device)
#fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
#fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "[MASK] yaptın?"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading weights from pretrained bert: dbmdz/bert-base-turkish-cased


BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, 

In [ ]:
val_loader = DataLoaderCustom(8, 512, device="cuda", split="train")

class weights 0:notNext, 1:isNext ->  tensor([0.2428, 0.7572], device='cuda:0')


In [ ]:
x_batch = val_loader.next_batch()

In [ ]:
type(x_batch)

dict

In [ ]:
w = model_from_hf(**x_batch)

In [ ]:
w.loss

tensor(8.6798, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
type(w)

model.BertForPreTrainingOutput

In [ ]:
import inspect

signature = inspect.signature(model_from_hf.forward)

# Parametreleri yazdır
for parametre in signature.parameters.values():
    print(parametre)

input_ids: torch.Tensor
attention_mask: torch.Tensor
token_type_ids: torch.Tensor
position_ids: torch.Tensor = None
labels: Optional[torch.Tensor] = None
next_sentence_label: Optional[torch.Tensor] = None
class_weights: Optional[torch.Tensor] = None


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [ ]:
tokenizer.is_fast

True

In [ ]:
tokenizer.slow_tokenizer_class

transformers.models.bert.tokenization_bert.BertTokenizer

In [ ]:
from transformers.models.bert.tokenization_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

pretokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), 
                                               pre_tokenizers.Digits(individual_digits=True),
                                               pre_tokenizers.Punctuation()])

In [2]:
pretokenizer.pre_tokenize_str("Merhaba [MASK] efendi nasıl gidiyor?")

[('Merhaba', (0, 7)),
 ('[', (8, 9)),
 ('MASK', (9, 13)),
 (']', (13, 14)),
 ('efendi', (15, 21)),
 ('nasıl', (22, 27)),
 ('gidiyor', (28, 35)),
 ('?', (35, 36))]